In [97]:
# Import dependencies
import numpy as np
import statsmodels.api as sm
import pandas as pd
import matplotlib.pyplot as plt
import datetime as dtm
import math
import scipy.stats as sps
from sklearn.model_selection import KFold
import seaborn as sns

In [98]:
#conda config --add channels conda-forge
#conda config --set channel_priority strict

In [99]:
#conda install -c conda-forge arch-py

In [100]:
#conda install arch-py

In [101]:
# Read the previously extracted financial data. 
df = pd.read_csv("/Users/hilalbuseakkas/Desktop/Imperial/Research Project/new data set copy/BTCUSD.csv")
df

,Date,Open*,High,Low,Close**,Volume,Market Cap
0,"Jul 22, 2022","$23,163.75","$23,671.93","$22,603.42","$22,714.98","$31,421,555,646","$433,866,028,177"
1,"Jul 21, 2022","$23,233.20","$23,388.32","$22,431.15","$23,164.63","$33,631,012,204","$442,435,421,339"
2,"Jul 20, 2022","$23,393.19","$24,196.82","$23,009.95","$23,231.73","$42,932,549,127","$443,696,738,856"
3,"Jul 19, 2022","$22,467.85","$23,666.96","$21,683.41","$23,389.43","$48,765,202,697","$446,690,803,766"
4,"Jul 18, 2022","$20,781.91","$22,633.03","$20,781.91","$22,485.69","$39,974,475,562","$429,412,153,274"
...,...,...,...,...,...,...,...
3368,"May 02, 2013",$116.38,$125.60,$92.28,$105.21,$0,"$1,168,517,495"
3369,"May 01, 2013",$139.00,$139.89,$107.72,$116.99,$0,"$1,298,954,594"
3370,"Apr 30, 2013",$144.00,$146.93,$134.05,$139.00,$0,"$1,542,813,125"
3371,"Apr 29, 2013",$134.44,$147.49,$134.00,$144.54,$0,"$1,603,768,865"


In [102]:
from arch import arch_model

In [103]:
# Get the relevant information
df_prices = df[['Date','Close**']]
df_prices.columns = ['Date','Prices']
df_prices

,Date,Prices
0,"Jul 22, 2022","$22,714.98"
1,"Jul 21, 2022","$23,164.63"
2,"Jul 20, 2022","$23,231.73"
3,"Jul 19, 2022","$23,389.43"
4,"Jul 18, 2022","$22,485.69"
...,...,...
3368,"May 02, 2013",$105.21
3369,"May 01, 2013",$116.99
3370,"Apr 30, 2013",$139.00
3371,"Apr 29, 2013",$144.54


In [104]:
# Reverse the data frame to have a proper chronological order. 
df_prices=df_prices.loc[::-1]
df_prices

,Date,Prices
3372,"Apr 28, 2013",$134.21
3371,"Apr 29, 2013",$144.54
3370,"Apr 30, 2013",$139.00
3369,"May 01, 2013",$116.99
3368,"May 02, 2013",$105.21
...,...,...
4,"Jul 18, 2022","$22,485.69"
3,"Jul 19, 2022","$23,389.43"
2,"Jul 20, 2022","$23,231.73"
1,"Jul 21, 2022","$23,164.63"


In [105]:
# Set the date as index to process each day. 
df_prices.set_index(['Date'], inplace=True)
df_prices

,Prices
Date,
"Apr 28, 2013",$134.21
"Apr 29, 2013",$144.54
"Apr 30, 2013",$139.00
"May 01, 2013",$116.99
"May 02, 2013",$105.21
...,...
"Jul 18, 2022","$22,485.69"
"Jul 19, 2022","$23,389.43"
"Jul 20, 2022","$23,231.73"


In [106]:
# Get rid of the commas & dollar signs. 
df_prices['Prices'] = df_prices['Prices'].str.replace('$', '')
df_prices['Prices'] = df_prices['Prices'].str.replace(',', '')
df_prices

/var/folders/5l/m0f_cfmj1zl083r_mtfh58f40000gn/T/ipykernel_26148/21575317.py:2: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df_prices['Prices'] = df_prices['Prices'].str.replace('$', '')
/var/folders/5l/m0f_cfmj1zl083r_mtfh58f40000gn/T/ipykernel_26148/21575317.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_prices['Prices'] = df_prices['Prices'].str.replace('$', '')
/var/folders/5l/m0f_cfmj1zl083r_mtfh58f40000gn/T/ipykernel_26148/21575317.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value 

,Prices
Date,
"Apr 28, 2013",134.21
"Apr 29, 2013",144.54
"Apr 30, 2013",139.00
"May 01, 2013",116.99
"May 02, 2013",105.21
...,...
"Jul 18, 2022",22485.69
"Jul 19, 2022",23389.43
"Jul 20, 2022",23231.73


In [107]:
# Convert the data to float. 
df_prices['Prices'] = df_prices['Prices'].astype(float)
df_prices['Prices']

/var/folders/5l/m0f_cfmj1zl083r_mtfh58f40000gn/T/ipykernel_26148/104500568.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_prices['Prices'] = df_prices['Prices'].astype(float)


Date
Apr 28, 2013      134.21
Apr 29, 2013      144.54
Apr 30, 2013      139.00
May 01, 2013      116.99
May 02, 2013      105.21
                  ...   
Jul 18, 2022    22485.69
Jul 19, 2022    23389.43
Jul 20, 2022    23231.73
Jul 21, 2022    23164.63
Jul 22, 2022    22714.98
Name: Prices, Length: 3373, dtype: float64

In [108]:
# Calculate the returns on a daily basis. 
df_prices['Returns'] = df_prices['Prices'].pct_change()*100


In [109]:
df_prices['Returns'].shift(1)

Date
Apr 28, 2013          NaN
Apr 29, 2013          NaN
Apr 30, 2013     7.696893
May 01, 2013    -3.832849
May 02, 2013   -15.834532
                  ...    
Jul 18, 2022    -1.939470
Jul 19, 2022     8.211762
Jul 20, 2022     4.019178
Jul 21, 2022    -0.674236
Jul 22, 2022    -0.288829
Name: Returns, Length: 3373, dtype: float64

In [110]:
np.log(df_prices['Returns'])

/Users/hilalbuseakkas/opt/anaconda3/lib/python3.9/site-packages/pandas/core/arraylike.py:364: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)
/Users/hilalbuseakkas/opt/anaconda3/lib/python3.9/site-packages/pandas/core/arraylike.py:364: RuntimeWarning: invalid value encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)


Date
Apr 28, 2013         NaN
Apr 29, 2013    2.040817
Apr 30, 2013         NaN
May 01, 2013         NaN
May 02, 2013         NaN
                  ...   
Jul 18, 2022    2.105568
Jul 19, 2022    1.391078
Jul 20, 2022         NaN
Jul 21, 2022         NaN
Jul 22, 2022         NaN
Name: Returns, Length: 3373, dtype: float64

In [111]:
df_prices.dropna(inplace = True)

In [112]:
am = arch_model(df_prices['Returns'])

In [113]:
res = am.fit(update_freq=5)
print(res.summary())

Iteration:      5,   Func. Count:     38,   Neg. LLF: 9249.534917903356
Iteration:     10,   Func. Count:     64,   Neg. LLF: 9226.765367637483
Optimization terminated successfully    (Exit mode 0)
            Current function value: 9226.765367637483
            Iterations: 11
            Function evaluations: 68
            Gradient evaluations: 11
                     Constant Mean - GARCH Model Results                      
Dep. Variable:                Returns   R-squared:                       0.000
Mean Model:             Constant Mean   Adj. R-squared:                  0.000
Vol Model:                      GARCH   Log-Likelihood:               -9226.77
Distribution:                  Normal   AIC:                           18461.5
Method:            Maximum Likelihood   BIC:                           18486.0
                                        No. Observations:                 3372
Date:                Wed, Jul 27 2022   Df Residuals:                     3371
Time:          

In [114]:
res = sm.tsa.ARMA(df_prices['Returns'], (1,1)).fit(disp=-1)

/Users/hilalbuseakkas/opt/anaconda3/lib/python3.9/site-packages/statsmodels/tsa/arima_model.py:472: FutureWarning: 
statsmodels.tsa.arima_model.ARMA and statsmodels.tsa.arima_model.ARIMA have
been deprecated in favor of statsmodels.tsa.arima.model.ARIMA (note the .
between arima and model) and
statsmodels.tsa.SARIMAX. These will be removed after the 0.12 release.

statsmodels.tsa.arima.model.ARIMA makes use of the statespace framework and
is both well tested and maintained.

To silence this warning and continue using ARMA and ARIMA until they are
removed, use:

import warnings
warnings.filterwarnings('ignore', 'statsmodels.tsa.arima_model.ARMA',
                        FutureWarning)
warnings.filterwarnings('ignore', 'statsmodels.tsa.arima_model.ARIMA',
                        FutureWarning)

  warnings.warn(ARIMA_DEPRECATION_WARN, FutureWarning)
/Users/hilalbuseakkas/opt/anaconda3/lib/python3.9/site-packages/statsmodels/tsa/base/tsa_model.py:524: ValueWarning: No frequency information

In [115]:
sm.stats.acorr_ljungbox(res.resid, lags=[1], return_df=True)

,lb_stat,lb_pvalue
1,0.000018,0.996587
